In [1]:
#https://scrapfly.io/blog/web-scraping-with-selenium-and-python/

TODO
- явно есть ошибки не корретного парсинга, можно найти кейсы по дешевым билетам
- сделать через распаралеривание (4-6 потоков)
- подумать может сделать без селениума
- на сервер запустить скрипт
- продумать как парсить билеты туда назад (тупо с двумя длинными списками будет некорректно)
- сохранять исходники страниц (или скрины)

In [2]:
from utils.aviasales_parser import parse_page, list_dates, get_ticket

In [3]:
from itertools import product

from tqdm import tqdm_notebook as tqdm

from datetime import datetime, timedelta
import time

import pandas as pd
import warnings
warnings.filterwarnings("ignore")



In [ ]:
origins =  ['BUD', 'TSR', 'OSI', 'PEV', 'SOB', 'DEB', 'ZAG', 'BEG', 'TGD'] #['LED', 'MOW', 'KZN']
destinations = ['HEL', 'TLL']# ['BEG', 'TGD', 'TIA', 'TIV']
range_date = ['2023-11-01', '2023-11-15']

date_start, date_end = range_date[0], range_date[1]
datetime_start = datetime.strptime(date_start, '%Y-%m-%d')
datetime_end = datetime.strptime(date_end, '%Y-%m-%d')
range_dates = list_dates(datetime_start, datetime_end)

parameters = list(product(origins, destinations, list_dates(datetime_start, datetime_end)))

for origin, destination, datetime_object in tqdm(parameters, total=len(parameters), desc='parse ticket'):
    get_ticket(origin, destination, datetime_object)
    
    
# origins = ['LED', 'MOW', 'KZN'] #
# destinations = ['BEG', 'TIV', 'TGD']
# range_date = ['2023-12-25', '2024-01-15']

    

# df = pd.DataFrame(tickets)
# df.to_csv('tickets.txt', sep = ';', index = False, encoding='utf-8-sig')

parse ticket:   0%|          | 0/270 [00:00<?, ?it/s]